In [1]:
import ee
import os
import geojson
import matplotlib.pyplot as plt
import numpy as np
import IPython.display as disp
from geemap import png_to_gif
from geemap.cartoee import get_map, add_gridlines, add_scale_bar_lite, add_north_arrow
import requests
ee.Initialize()
%matplotlib inline

In [2]:
os.listdir("Locations/")

['algeciras.geojson',
 'gibraltar.geojson',
 'gibraltar_peq.geojson',
 'map.geojson',
 'shangai.geojson',
 'south_sicilia.geojson']

In [21]:
filename = "ferrol.geojson"
with open("Locations\\" + filename) as f:
    gj = geojson.load(f)

coords = gj['features'][0]['geometry']['coordinates']
aoi = ee.Geometry.Polygon(coords)

start_date = "2021-12-01" # 
end_date = "2021-12-31" #

ffa_db = (ee.ImageCollection("COPERNICUS/S2_SR")
                       .filterBounds(aoi)
                       .filterDate(ee.Date(start_date), ee.Date(end_date))
                       .sort('system:time_start'))

#import time
#acq_times = ffa_db.aggregate_array('system:time_start').getInfo()
#[time.strftime('%x', time.gmtime(acq_time/1000)) for acq_time in acq_times]

im_list = ffa_db.toList(ffa_db.size())
index = 4

url = ee.Image(im_list.get(index)).select("B4", "B3", "B2").clip(aoi).getThumbURL({'min': 0, 'max': 3000})
disp.Image(url=url, width=1200)

In [22]:

img_data = requests.get(url).content
with open('Images/ferrol.png', 'wb') as handler:
    handler.write(img_data)